<a href="https://colab.research.google.com/github/DrFrank25/Syndecan_4-Ag73/blob/main/HBonds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Script Hydrogen Bonds (H-bonds) plot**

In [ ]:
# --- Import libraries ---
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
import pandas as pd

In [ ]:
# ===== Settings =====
COLORS = ['#1f77b4', '#ff7f0e', '#2ca02c']
FIGSIZE = (11, 6)
DPI = 600
LEGEND_NAMES = ["Run 1", "Run 2", "Run 3"]

In [ ]:
# ===== File upload =====
uploaded = files.upload()
if not uploaded:
    raise SystemExit("No file uploaded. Please upload .xvg files.")


In [ ]:
# ===== Function to read .xvg data =====
def read_xvg_from_bytes(content_bytes, encoding='utf-8'):
    s = content_bytes.decode(encoding, errors='replace')
    times, vals = [], []
    for line in s.splitlines():
        line = line.strip()
        if not line or line.startswith(('#', '@')):
            continue
        parts = line.split()
        if len(parts) >= 2:
            try:
                times.append(float(parts[0]))
                vals.append(float(parts[1]))
            except ValueError:
                continue
    return np.array(times), np.array(vals)

plt.figure(figsize=FIGSIZE)


In [ ]:
# Loop over uploaded files
for i, (name, content) in enumerate(uploaded.items()):
    time, hbonds = read_xvg_from_bytes(content)

    if time.size == 0 or hbonds.size == 0:
        print(f"Warning: '{name}' contains no valid data. Skipping.")
        continue

    # Sort by time
    order = np.argsort(time)
    time = time[order]
    hbonds = hbonds[order]

    # Mask valid data
    mask_valid = np.isfinite(time) & np.isfinite(hbonds) & (time >= 0) & (hbonds >= 0)
    time_masked = np.where(mask_valid, time, np.nan)
    hbonds_masked = np.where(mask_valid, hbonds, np.nan)

    # Statistics
    mean_val = np.nanmean(hbonds_masked)
    std_val = np.nanstd(hbonds_masked)

    color = COLORS[i % len(COLORS)]
    curve_name = LEGEND_NAMES[i] if i < len(LEGEND_NAMES) else f"Run {i+1}"

    # Raw curve
    valid_raw = np.isfinite(time_masked) & np.isfinite(hbonds_masked)
    plt.plot(time_masked[valid_raw], hbonds_masked[valid_raw],
             linewidth=1.0, label=f"{curve_name}", color=color)

    print(f"{curve_name}:")
    print(f"  Mean: {mean_val:.2f} H-bonds")
    print(f"  Standard deviation: {std_val:.2f} H-bonds\n")

# Axes configuration
plt.xlabel("Time (ps)", fontsize=12)
plt.ylabel("No. of hydrogen bonds", fontsize=12)
plt.tick_params(axis='both', which='major', labelsize=14)

# Custom legend
plt.legend(
    loc='upper right',
    fontsize=10,
    frameon=True,
    facecolor='white',
    edgecolor='white',
    fancybox=False,
    shadow=False
)

plt.grid(False)
plt.tight_layout()

# --- save and render the figure (at the end of the script, BEFORE plt.show()) ---
fig = plt.gcf()  # get the current figure (the one just plotted)
fig.tight_layout()
fig.canvas.draw()  # force rendering in Colab
plt.show()

In [ ]:
# --- Save and download the figure ---
fig = plt.gcf()  # get current figure
fig.tight_layout()
fig.canvas.draw()

fig.savefig(
    "HBonds.png",
    dpi=1200,
    bbox_inches='tight',
    transparent=False
)

files.download("HBonds.png")

In [ ]:
# ======================
# Overall mean of the 3 runs (raw values)
# ======================

all_values = []

for name, content in uploaded.items():
    time, hbonds = read_xvg_from_bytes(content)
    mask_valid = np.isfinite(hbonds) & (hbonds >= 0)
    if np.any(mask_valid):
        all_values.extend(hbonds[mask_valid])

all_values = np.array(all_values)

global_mean = np.mean(all_values)
global_std = np.std(all_values)

print("\n=== Global mean statistics (raw data) ===")
print(f"Global mean: {global_mean:.4f} H-bonds")
print(f"Global standard deviation: {global_std:.4f} H-bonds")